Notebook by:

* Lorenzo Pannacci 1948926
* Stefano Rinaldi 1945551

## Startup

In [ ]:
######################
# LIBRARIES DOWNLOAD #
######################

install_packages = True
if install_packages:
    %pip install numpy pandas

In [ ]:
####################
# LIBRARIES IMPORT #
####################

import numpy as np
import pandas as pd

## 1 - Recommendation sytem 


### 1.1 - top 10 films


for this exercise I did not understand well if I have to consider only the clicks or also the generes of a film. Either way, I will propose both solutions.


### solution 1
**taking into considetation only click counts**

This is what I am going to do steps to do: 

1) Load the data <br>
2) Fix the dataset <br>
3) Group by user and movie <br>
4) Count the clicks <br>
5) Sort the data by click count for each user.<br>
6) Extract the top 10 movies for each user.<br>



In [ ]:
# 1)/2) load the data and fix the dataset
dataset = pd.read_csv('vodclickstream_uk_movies_03.csv')

#clean it
dataset = dataset.drop("Unnamed: 0", axis=1)
#Convert genres from comma separated value to a list of values
dataset["genres_list"] = dataset.genres.apply(lambda row: row.split(','))

In [ ]:
#dataset.datetime = pd.to_datetime(dataset.datetime, errors='coerce')
#dataset.release_date = pd.to_datetime(dataset.release_date, errors='coerce')

#identify unique values:
#unique_genres = set()
#dataset['genres_list'].apply(lambda row: [unique_genres.add(value) for value in row])
#unique_genres

In [ ]:
# 3)/4)/5)/6) group by user and movie, count clicks and sort them

#create a new dataframe with the information I need
users = dataset[["genres_list", "user_id","title", "genres"]]

#group user and movie and count the click
user_movie_counts =users.groupby(['user_id', 'title']).size().reset_index(name='click_count')

#sort them
user_movie_counts = user_movie_counts.sort_values(by=['user_id', 'click_count'], ascending=[True, False])

#take the top 10 movies for each user
user_movie_counts = user_movie_counts.groupby('user_id').head(10)

In [ ]:
#return a new dataframe with the top 10 movies for each user;
# I give as information the user, the titple and the genre of the film (as well as the click count)
#other pieces of information are not asked

result = pd.merge(user_movie_counts, users[['user_id', 'title', 'genres']], on=['user_id', 'title'], how='left')

result['genres'] = result['genres'].astype(str)

#remove duplicates. I have to do this because the previous code gives the right answer, but
#it creates n rows for the x value of the click count.
#so if a film has 9 clicks, it creates 9 times that rows.
result = result.drop_duplicates()

print(result)

           user_id              title  click_count  \
0       00004e2862           Hannibal            1   
1       000052a0a0             Looper            9   
10      000052a0a0            Frailty            3   
13      000052a0a0            Jumanji            3   
16      000052a0a0      Resident Evil            2   
...            ...                ...          ...   
613980  fffeac83be        To the Bone            1   
613981  fffeac83be         True Story            1   
613982  ffff2c5f9e  Forks Over Knives            1   
613983  ffff2c5f9e           Hot Fuzz            1   
613984  ffffd36adf   Dead Man Walking            1   

                                    genres  
0                   Crime, Drama, Thriller  
1          Action, Drama, Sci-Fi, Thriller  
10                  Crime, Drama, Thriller  
13      Adventure, Comedy, Family, Fantasy  
16                  Action, Horror, Sci-Fi  
...                                    ...  
613980                              

### solution 2
**take into consideration both clicks and genres of a film**

In this case, I will extract the top 10 films by clicks from user, but then I will sort those films not only on the bases of the clicks but also on the genres it has, taking into considerations how many times a user saw a particular genre (still in the temporary top 10 films), giving to it priority. <br>
The first filter will be the click-counts but then I will apply also a "genre" filter. This is also done because a lot of films have the same amount of clicks, so how can I order them giving them priority? I do this by using the genre of the film. To sum up, if a genre (such as "Drama") has been clicked by the user 10 times and another one ("Comedy"), has been clicked 2 times, two films with the same amounts of clicks, will be ordered by the score of their genre, such that "Drama" in this case will have priority. <br>

steps to folow for this scenario: <br>

7) divide the "genre" column <br>
8) count how many times a genre appear for each user <br>
9) merge the result with the previous dataset <br>
10) sum the clicks and the genre to have the score of the film <br>



In [ ]:
####the code above is still the same, you have to add only these chunks here:
# 7)/8) extract the genres from the column, specifically for those films with more than one genre. count the occurrences

#take genres
flat_genres = result['genres'].explode().str.split(',').explode().str.strip()

#merge with the user_id 
flat_genres_with_user = pd.concat([result['user_id'], flat_genres], axis=1)

#count the occurrences of each genre for each user
genre_counts = flat_genres_with_user.value_counts().reset_index(name='genre_count')

           user_id    genres  genre_count
0       68768e88bf  Thriller           10
1       b35d4891af  Thriller           10
2       4b44d7203d    Action           10
3       60716debd8    Comedy           10
4       ecab823e36    Comedy           10
...            ...       ...          ...
847196  64a9e13237  Thriller            1
847197  64aa0228d8     Drama            1
847198  64aa0228d8   Romance            1
847199  64aa4cfed1    Action            1
847200  ffffd36adf     Drama            1

[847201 rows x 3 columns]


In [ ]:
#9) merge the result

result['genres'] = result['genres'].str.split(', ')

#Merge. Here I have the count of genres for each author
merged_df = pd.merge(result.explode('genres'), genre_counts, on=['user_id', 'genres'], how='left')

#merge. Here I have the "genre_score" for each film of each author.
grouped_df = merged_df.groupby(['user_id', 'title', 'click_count']).agg({'genre_count': 'sum'}).reset_index()

#merge
result = pd.merge(result, grouped_df[['user_id', 'title', 'genre_count']], on=['user_id', 'title'], how='left')

           user_id              title  click_count  \
0       00004e2862           Hannibal            1   
1       000052a0a0             Looper            9   
2       000052a0a0            Frailty            3   
3       000052a0a0            Jumanji            3   
4       000052a0a0      Resident Evil            2   
...            ...                ...          ...   
447109  fffeac83be        To the Bone            1   
447110  fffeac83be         True Story            1   
447111  ffff2c5f9e  Forks Over Knives            1   
447112  ffff2c5f9e           Hot Fuzz            1   
447113  ffffd36adf   Dead Man Walking            1   

                                      genres  genre_count  
0                   [Crime, Drama, Thriller]            3  
1          [Action, Drama, Sci-Fi, Thriller]           18  
2                   [Crime, Drama, Thriller]           11  
3       [Adventure, Comedy, Family, Fantasy]            9  
4                   [Action, Horror, Sci-Fi]       

In [ ]:
#10) get the new score and sort the result
import math
#sum the clicks and the genres (final score)
result['importance'] = result['click_count'] + result['genre_count']

#sort the results 
result = result.sort_values(by=['user_id', 'importance'], ascending=[True, False])

print(result)

           user_id                       title  click_count  \
0       00004e2862                    Hannibal            1   
1       000052a0a0                      Looper            9   
10      000052a0a0               The Nice Guys            1   
8       000052a0a0  Resident Evil: Retribution            1   
5       000052a0a0                     Ant-Man            1   
...            ...                         ...          ...   
447101  fffeac83be                 Amanda Knox            2   
447108  fffeac83be    The Paedophile Next Door            1   
447112  ffff2c5f9e                    Hot Fuzz            1   
447111  ffff2c5f9e           Forks Over Knives            1   
447113  ffffd36adf            Dead Man Walking            1   

                                            genres  genre_count  importance  
0                         [Crime, Drama, Thriller]            3           4  
1                [Action, Drama, Sci-Fi, Thriller]           18          27  
10      [

**comment**

We can see that the results slightly changes. The top 10 movies are exactly the same as before because the prime filter is the *click-count* (which remains invariated). Yet, the final order is different as in this case we take into account also the genre preferences of a user. Taking as an example *000052a0a0* user, the top 10 of films between only films and both films and genre is different: <br>
- **Looper** still dominates as it is the one with both most clicks and with genres that are liked (or at least seen) the most by the user. <br>
- However, when we get to the second position, we can immediatly see a difference: for genre and clicks now **The Nice Guys** is the second best guess whereas on the original top list it was **Frailty**. <br>

### 1.2 Minhash Signatures

steps to follow: <br>
1) **take the data**: <br>
extract the data I need ("user_id" and "genres" columns)
2) **store users**: <br>
I put into a dictionary all genres associated with a user.
3) **create hash function**: <br>
I create the hash function by creating my own code as it is **not permitted** to use **already implemented functions**.
4) **calculate the minhash**: <br>
Now I find the minhas for genres and I store the values into a dictionary.
5) **create the buckets**: <br>
I create another dictionary in order to store users based on their minhash.Users with the same MinHash signature are grouped into the same bucket.

In [ ]:
#1) retrieve useful data
data = dataset[["user_id", "genres"]]

data = data.sort_values(by=['user_id'], ascending=[True])

In [ ]:
#2) store users

#store in this dictionary the user, and the genre they watched (liked)
user_sets = {}

#for each row od the dataframe:
for index, row in data.iterrows():
    #take the value in user_id column 
    user_id = row['user_id']
    #take the value(s) in genre column
    genres = set(row['genres'].split(', '))
    
    if user_id not in user_sets:
        user_sets[user_id] = genres
    else:
        #if the user_id already exist then append the genres
        user_sets[user_id].update(genres)


In [ ]:
#3) create my hash function   

def my_hash(input_string):
    ##random number
    hash_value = 13
    
    for char in input_string:
        #other random number + univocate code for each character
        hash_value = (hash_value * 17 + ord(char)) % (2**64)

    return hash_value

In [ ]:
#4) calculate minhash

#store the minhash. Here I have the minhash and the users associated with it
minhash_signatures = {}

for user_id, genres_set in user_sets.items():
    #store hash values for the genre
    hash_values = []

    #calculate hash value for genre
    for genre in genres_set:
        #append it
        hash_values.append(my_hash(genre))

    #search the minimun
    minhash_value = min(hash_values)

    #stor it for each user
    minhash_signatures[user_id] = [minhash_value]


In [ ]:
#5) bucket
#group similar users into the sam bucket
buckets = {}

# Iterate through users and their MinHash signatures
for user_id, signature in minhash_signatures.items():
    # Convert the signature list to a tuple to use it as a dictionary key
    signature_key = tuple(signature)

    # Add the user to the corresponding bucket
    if signature_key in buckets:
        buckets[signature_key].append(user_id)
    else:
        buckets[signature_key] = [user_id]

### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at __most five movies__ to the user to watch based on the movies clicked by similar users. 

To recommend at most five movies given a user_id, use the following procedure: 

1. Identify the <ins>two most similar</ins> users to this user.
2. If these two users have any movies __in common__, recommend those movies based on the total number of clicks by these users.
3. If there are __no more common__ movies, try to propose the most clicked movies by the __most similar user first__, followed by the other user. 

__Note:__ At the end of the process, we expect to see at most five movies recommended to the user.

__Example:__ assume you've identified user __A__ and __B__ as the most similar users to a single user, and we have the following records on these users: 

- User A with 80% similarity
- User B with 50% similarity
  
|user|movie title|#clicks|
|---|---|---|
|A|Wild Child|20|
|A|Innocence|10|
|A|Coin Heist|2|
|B|Innocence|30|
|B|Coin Heist|15|
|B|Before I Fall|30|
|B|Beyond Skyline|8|
|B|The Amazing Spider-Man|5|

- __Recommended movies__ in order:
    - Innocence
    - Coin Heist
    - Wild Child
    - Before I Fall
    - Beyond Skyline

In [ ]:
# TODO

## 2. Grouping Users together!

Now, we will deal with clustering algorithms that will provide groups of Netflix users that are similar among them.

To solve this task, you must accomplish the following stages:

### 2.1 Getting your data + feature engineering

1)  Access to the data found in [this dataset](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies)

2)  Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as *feature engineering*. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features **for each** user:

    a)  Favorite genre (i.e., the genre on which the user spent the most time)

    b)  Average click duration

    c)  Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)

    d)  Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?

    e)  Average time spent a day by the user (considering only the days he logs in)

So, in the end, you should have for each user_id five features.

3)  Consider at least 10 additional features that can be generated for each user_id (you can use chatGPT or other LLM tools for suggesting features to create). Describe each of them and add them to the previous dataset you made (the one with five features). In the end, you should have for each user at least 15 features (5 recommended + 10 suggested by you).

In [ ]:
# TODO

### 2.2 Choose your features (variables)!

You may notice that you have plenty of features to work with now. So, it would be best to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

1)  *To normalise or not to normalise? That's the question*. Sometimes, it is worth normalizing (scaling) the features. Explain if it is a good idea to perform any normalization method. If you think the normalization should be used, apply it to your data (look at the available normalization functions in the `scikit-learn` library).

2)  Select **one** method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

In [ ]:
# TODO

### 2.3 Clustering!

1)  Implement the K-means clustering algorithm (**not** ++: random initialization) using MapReduce. We ask you to write the algorithm from scratch following what you learned in class.

2)  Find an optimal number of clusters. Use at least two different methods. If your algorithms provide diverse optimal K's, select one of them and explain why you chose it.

3)  Run the algorithm on the data obtained from the dimensionality reduction.

4)  Implement **K-means++** from scratch and explain the differences with the results you got earlier.

5)  Ask ChatGPT to recommend other clustering algorithms and choose one. Explain your choice, then ask ChatGPT to implement it or use already implemented versions (e.g., the one provided in the scikit-learn library) and run it on your data. Explain the differences (if there are any) in the results. Which one is the best, in your opinion, and why?

In [ ]:
# TODO

### 2.4 Analysing your results!

You are often encouraged to explain the main characteristics that your clusters have. The latter is called the *Characterizing Clusters* step. Thus, follow the next steps to do it:

1)  Select 2-3 variables you think are relevant to identify the cluster of the customer. For example, Time_Day, Average Click Duration, etc.

2)  Most of your selected variables will be numerical (continuous or discrete), then categorize them into four categories.

3)  With the selected variables, perform pivot tables. On the horizontal axis, you will have the clusters, and on the vertical axis, you will have the categories of each variable. Notice that you have to do one pivot table per variable.

4)  Calculate the percentage by column for each pivot table. The sum of each row (cluster) must be 100.

5)  Interpret the results for each pivot table.

6)  Use any known metrics to estimate clustering algorithm performance (how good are the clusters you found?). Comment on the results obtained.

In [ ]:
# TODO

## 3. Bonus Question

We remind you that we consider and grade the bonuses only if you complete the entire assignment.

[Density-based clustering](https://wires.onlinelibrary.wiley.com/doi/epdf/10.1002/widm.30) identifies clusters as regions in the data space with high point density that are separated from other clusters by regions of low point density. The data points in the separating regions of low point density are typically considered noise or outliers. Typical algorithms that fall into this category are [OPTICS](https://dl.acm.org/doi/pdf/10.1145/304181.304187) and [DBSCAN](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf).

1)  Ask ChatGPT (or any other LLM tool) to list three algorithms for Density-Based Clustering. Choose one and use it on the same dataset you used in 2.3. Analyze your results: how different are they from the centroid-based version?

__Note__: You can implement your algorithm from scratch or use the one implemented in the scikit-learn library; the choice is up to you!

In [ ]:
# TODO

## 4. Command Line Question

Here is another command line question to enjoy. We previously stated that using the command line tools is a skill that Data Scientists must master.

In this question, you should use any command line tool that you know to answer the following questions using the same dataset that you have been using so far:
  + What is the most-watched Netflix title?
  + Report the average time between subsequent clicks on Netflix.com
  + Provide the ID of the user that has spent the most time on Netflix
    
__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.  

In [ ]:
# TODO

## 5. Algorithmic Question 

Federico studies in a demanding university where he has to take a certain number $N$ of exams to graduate,  but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system. 

He was given an initial personal score of $S$ when he enrolled, which changes every time he takes an exam: now comes the crazy part.
He soon discovered that every of the $N$ exams he has to take is assigned a mark $p$. Once he has chosen an exam, his score becomes equal to the mark $p$, and at the same time, the scoring system changes:
+ If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity $S - p$.
+ If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity $p - S$.
  
So, for example, consider $S=8$ as the initial personal score. Federico must decide which exam he wants to take, being $[5,7,1]$ the marks list. If he takes the first one, being $5 < 8$ and $8 - 5 = 3$, the remaining list now becomes $[10,4]$, and his score is updated as $S = 5$.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line,  and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get. 

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.

b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university! 

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

Here are some input/output examples (the first value is the initial personal score, and the second line contains the list of marks): 

__Input 1__
```
8
5 7 1 
```

__Output 1__
```
11
```

__Input 2__
```
25
18 24 21 32 27
```

__Output 2__
```
44
```

__Input 3__
```
30
13 27 41 59 28 33 39 19 52 48 55 79
```

__Output 3__
```
109
```

In [ ]:
# TODO